# Notebook 2: MED-PC Calculating Latencies

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from medpc2excel.medpc_read import medpc_read

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

In [5]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (10,6)

# Loading in Recording and Metadata from Previous Notebook

# NOTE: If you changed the directory of where the MED-PC recording dataframes, then you must change the directories in the following paths

- This will get all the files in each specified path

In [6]:
concatted_medpc_files = glob.glob("./proc/extracted_recording_data_and_metadata/*/MEDPC_recording_cage_*.csv")
metadata_files = glob.glob("./proc/extracted_recording_data_and_metadata/*/metadata_cage_*.csv")
recording_and_metadata_files = glob.glob("./proc/extracted_recording_data_and_metadata/*/recording_metadata_cage_*.csv")

# NOTE: If there is more than one MED-PC recording dataframes, then you must manually change the path in the `pd.read_csv()` for the corresponding variables. Verify if this is the correct file that you want to use

In [7]:
metadata_files[0]

'./proc/extracted_recording_data_and_metadata\\experiment_Pilot_of_Pilot_cage_1_2_date_20220504_20220516\\metadata_cage_1_2_date_20220504_20220516.csv'

In [8]:
concatted_medpc_files[0]

'./proc/extracted_recording_data_and_metadata\\experiment_Pilot_of_Pilot_cage_1_2_date_20220504_20220516\\MEDPC_recording_cage_1_2_date_20220504_20220516.csv'

In [9]:
recording_and_metadata_files[0]

'./proc/extracted_recording_data_and_metadata\\experiment_Pilot_of_Pilot_cage_1_2_date_20220504_20220516\\recording_metadata_cage_1_2_date_20220504_20220516.csv'

In [10]:
metadata_df = pd.read_csv(metadata_files[0], index_col=0)
concatted_medpc_df = pd.read_csv(concatted_medpc_files[0], index_col=0)
recording_and_metadata_df = pd.read_csv(recording_and_metadata_files[0], index_col=0)

- The Dataframe the contains the metadata(columns) of each recording file(rows)
    - We will mostly use this to get the cage number for each subject

In [11]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
0,./example_recordings\2022-05-04_08h42m_Subject...,C:\MED-PC\Data\2022-05-04_08h42m_Subject 1.1.txt,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
1,./example_recordings\2022-05-04_08h43m_Subject...,C:\MED-PC\Data\2022-05-04_08h43m_Subject 1.4.txt,05/04/22,05/04/22,1.4,Pilot of Pilot,Cage 1,4,08:43:55,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
2,./example_recordings\2022-05-04_08h43m_Subject...,C:\MED-PC\Data\2022-05-04_08h43m_Subject 2.2.txt,05/04/22,05/04/22,2.2,Pilot of Pilot,Cage 1,2,08:43:01,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
3,./example_recordings\2022-05-04_08h43m_Subject...,C:\MED-PC\Data\2022-05-04_08h43m_Subject 2.3.txt,05/04/22,05/04/22,2.3,Pilot of Pilot,Cage 1,3,08:43:11,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
4,./example_recordings\2022-05-04_10h11m_Subject...,C:\MED-PC\Data\2022-05-04_10h11m_Subject 1.2.txt,05/04/22,05/04/22,1.2,Pilot of Pilot,Cage 2,3,10:11:55,11:18:44,levelNP_CS_reward_laserepochON1st_noshock,2


- The Dataframe the contains the recording data for all the files.
    - The 1st row is the first data entries for each type of data(with the type being specified by the column label). This does not mean that all these are related or occured for the same trial. i.e. There can be many port entries and exits for a subject before and after a tone is played.
    - Each recording session will usually have a few thousand rows(although, most of of the columns will probably be blank by the end). And then after the last row of one session, the next row will be the next session with a different `file_path`

In [12]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,file_path
0,25.26,64.0,399.0,0.0,60.01,25.30,1.0,0.0,20220504,1.1,./example_recordings\2022-05-04_08h42m_Subject...
1,71.25,144.0,399.0,0.0,140.01,71.81,1.0,0.0,20220504,1.1,./example_recordings\2022-05-04_08h42m_Subject...
2,71.88,234.0,399.0,0.0,230.01,71.99,1.0,0.0,20220504,1.1,./example_recordings\2022-05-04_08h42m_Subject...
3,104.43,314.0,399.0,0.0,310.01,104.45,1.0,0.0,20220504,1.1,./example_recordings\2022-05-04_08h42m_Subject...
4,134.46,389.0,399.0,0.0,385.01,134.67,1.0,0.0,20220504,1.1,./example_recordings\2022-05-04_08h42m_Subject...


- This dataframe combines the recording data and metadata dataframes. This is done by using the subject ID as a common column to merge together off of.

In [13]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
0,25.26,64.0,399.0,0.0,60.01,25.30,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
1,71.25,144.0,399.0,0.0,140.01,71.81,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
2,71.88,234.0,399.0,0.0,230.01,71.99,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
3,104.43,314.0,399.0,0.0,310.01,104.45,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
4,134.46,389.0,399.0,0.0,385.01,134.67,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1


# Getting the Cage Numbers and the Dates for the file names

- Seeing if any rows have NaN values for the cages

In [14]:
recording_and_metadata_df[pd.isna(recording_and_metadata_df["cage"])]

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage


- Dropping the NaN values so we can get the cage numbers

In [15]:
recording_and_metadata_df = recording_and_metadata_df.dropna(subset=["cage"])

# Adjusting the days and cages that are being looked at

- List of all the subject names

In [16]:
all_subjects = sorted(recording_and_metadata_df["subject"].unique())

In [17]:
all_subjects

[1.1, 1.2, 1.3, 1.4, 2.1, 2.2, 2.3, 2.4]

- List of all the cage numbers

In [18]:
all_cages = sorted(recording_and_metadata_df["cage"].unique())

In [19]:
all_cages

[1, 2]

- List of all the dates

In [20]:
all_dates = sorted(recording_and_metadata_df["date"].unique())

In [21]:
all_dates

[20220504,
 20220505,
 20220506,
 20220507,
 20220508,
 20220509,
 20220510,
 20220511,
 20220512,
 20220516]

# NOTE: Follow directions below if you want to change which cages or dates to plot

- You must change the cell below by clicking on the cell and then pressing the `esc` key. Then press the `Y` button on your keyboard. If you want to switch it back do the same but press the `R` button instead. This switches it from a cell that is run as if it's code, or back to a cell where it's just treated as text
- Then from above, copy the numbers that you want to use for the cage numbers and the dates

# Getting the Cage Numbers and the Dates to name the files

In [22]:
# removing blank spaces
cage_numbers = [str(int(number)) for number in metadata_df["cage"].unique() if number is not np.nan]
# sorting numbers
cage_numbers = sorted(cage_numbers)
cage_numbers_for_title = "_".join(cage_numbers)

In [23]:
cage_numbers_for_title

'1_2'

In [24]:
# Getting the first and last recording date to get a range
earliest_date = concatted_medpc_df["date"].min()
latest_date = concatted_medpc_df["date"].max()

In [25]:
earliest_date

20220504

In [26]:
latest_date

20220516

# Getting the times that the tones are being played

- Getting all the rows that correspond to one of the files paths. This will be the equivalent of getting all the rows for one session

In [27]:
example_file = recording_and_metadata_df["file_path"].unique()[0]

In [28]:
example_file

'./example_recordings\\2022-05-04_08h42m_Subject 1.1.txt'

In [29]:
example_one_session_recording_df = recording_and_metadata_df[recording_and_metadata_df["file_path"] == example_file]

- Everything looks normal enough

In [30]:
example_one_session_recording_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
0,25.26,64.0,399.0,0.0,60.01,25.30,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
1,71.25,144.0,399.0,0.0,140.01,71.81,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
2,71.88,234.0,399.0,0.0,230.01,71.99,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
3,104.43,314.0,399.0,0.0,310.01,104.45,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
4,134.46,389.0,399.0,0.0,385.01,134.67,1.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1


- `(S)CSpresentation` is the time that the tone starts playing. MED-PC starts inputting in random numbers that are in the 1000's. So we will remove these

In [31]:
example_one_session_recording_df[50:60]

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
50,330.80,NaN,NaN,0.0,10000.0,333.45,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
51,333.49,NaN,NaN,0.0,9000.0,334.02,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
52,334.04,NaN,NaN,0.0,12000.0,334.35,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
53,334.57,NaN,NaN,0.0,9000.0,334.77,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
54,334.94,NaN,NaN,0.0,8500.0,336.06,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
55,336.65,NaN,NaN,0.0,9000.0,337.01,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
56,337.03,NaN,NaN,0.0,9500.0,337.27,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
57,337.29,NaN,NaN,0.0,12000.0,337.37,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
58,337.70,NaN,NaN,0.0,8000.0,341.49,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
59,350.30,NaN,NaN,0.0,9500.0,350.36,0.0,0.0,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1


- `(S)CSpresentation` then starts adding blanks(NaN) so we will remove these too

In [32]:
example_one_session_recording_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(B)shockintensity,date,subject,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
2536,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
2537,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
2538,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
2539,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1
2540,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,20220504,1.1,...,05/04/22,05/04/22,1.1,Pilot of Pilot,Cage 1,1,08:42:53,09:54:22,levelNP_CS_reward_laserepochON1st_noshock,1


- We will use a function that removes the `NaN` and the tone times that are divisible by 1000. This will get the tones time that were actually used for the recordings

In [33]:
example_one_session_valid_tones = processing.tone.get_valid_tones(example_one_session_recording_df["(S)CSpresentation"])

In [34]:
example_one_session_valid_tones

0       60.01
1      140.01
2      230.01
3      310.01
4      385.01
5      485.01
6      580.01
7      670.01
8      750.01
9      840.01
10     940.01
11    1030.01
12    1150.01
13    1240.01
14    1325.01
15    1415.01
16    1510.01
17    1630.01
18    1710.01
19    1805.01
20    1885.01
21    1975.01
22    2055.01
23    2130.01
24    2230.01
25    2325.01
26    2415.01
27    2495.01
28    2585.01
29    2685.01
30    2775.01
31    2895.01
32    2985.01
33    3070.01
34    3160.01
35    3255.01
36    3345.01
37    3425.01
38    3515.01
39    3615.01
40    3705.01
Name: (S)CSpresentation, dtype: float64

# Getting the First Port Entry After Each Tone

- With these tone playing times, we will get the first port entry that comes after the tone playing time
    - This was done by getting all the port entries that came after the tone. And then getting the port entry time that was earliest in time(aka smallest number)

In [35]:
processing.tone.get_first_port_entries_after_tone(tone_pd_series=example_one_session_valid_tones, port_entries_pd_series=example_one_session_recording_df["(P)Portentry"], port_exits_pd_series=example_one_session_recording_df["(N)Portexit"])

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone
0,60.01,71.25,71.81
1,140.01,145.56,145.65
2,230.01,250.18,250.55
3,310.01,311.58,311.87
4,385.01,386.07,386.16
5,485.01,504.87,505.17
6,580.01,584.42,584.51
7,670.01,677.80,678.43
8,750.01,753.33,753.52
9,840.01,845.38,845.66


- Getting the first port entry times for all the sessions

In [36]:
concatted_first_porty_entry_dataframe = processing.tone.get_concatted_first_porty_entry_after_tone_dataframe(concatted_medpc_df=concatted_medpc_df)

In [37]:
concatted_first_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject
0,60.01,71.25,71.81,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
1,140.01,145.56,145.65,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
2,230.01,250.18,250.55,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
3,310.01,311.58,311.87,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
4,385.01,386.07,386.16,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1


In [38]:
concatted_first_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject
0,60.01,71.25,71.81,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
1,140.01,145.56,145.65,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
2,230.01,250.18,250.55,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
3,310.01,311.58,311.87,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
4,385.01,386.07,386.16,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1


# Getting the Last Port Entry Before the Tone

- We will do the same, but get the port entries that happened before the tone

In [39]:
port_entries_pd_series = example_one_session_recording_df["(P)Portentry"]
port_exit_pd_series = example_one_session_recording_df["(N)Portexit"]

- Example run for one session

In [40]:
example_last_port_entries_before_tone = processing.tone.get_last_port_entries_before_tone(tone_pd_series=example_one_session_valid_tones, port_entries_pd_series=port_entries_pd_series, port_exits_pd_series=port_exit_pd_series)

In [41]:
example_last_port_entries_before_tone

,current_tone_time,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone
0,60.01,25.26,25.30
1,140.01,137.81,139.58
2,230.01,221.72,221.91
3,310.01,307.16,308.42
4,385.01,384.48,384.67
5,485.01,480.13,480.85
6,580.01,577.84,578.51
7,670.01,665.16,666.22
8,750.01,746.53,747.11
9,840.01,834.64,834.80


- Getting all the port entries that happen before the tone for all sessions

In [42]:
concatted_last_porty_entry_dataframe = processing.tone.get_concatted_last_porty_entry_before_tone_dataframe(concatted_medpc_df=concatted_medpc_df)

In [43]:
concatted_last_porty_entry_dataframe

,current_tone_time,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,file_path,date,subject
0,60.01,25.26,25.30,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
1,140.01,137.81,139.58,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
2,230.01,221.72,221.91,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
3,310.01,307.16,308.42,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
4,385.01,384.48,384.67,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1
...,...,...,...,...,...,...
3043,3255.01,3252.87,3253.33,./example_recordings\2022-05-16_11h28m_Subject...,20220516,2.4
3044,3345.01,3344.92,3362.29,./example_recordings\2022-05-16_11h28m_Subject...,20220516,2.4
3045,3425.01,3422.82,3423.22,./example_recordings\2022-05-16_11h28m_Subject...,20220516,2.4
3046,3515.01,3514.81,3533.87,./example_recordings\2022-05-16_11h28m_Subject...,20220516,2.4


## Merging the dataframes for the first port entry after the tone and the last tone before the tone 

In [44]:
# Combining it based on the tone times and the file path
concatted_first_and_last_porty_entry_dataframe = pd.merge(concatted_first_porty_entry_dataframe, concatted_last_porty_entry_dataframe,  how='left', left_on=['current_tone_time','file_path'], right_on = ['current_tone_time','file_path'], suffixes=('', '_y'))
# Dropping all columns that are duplicates
concatted_first_and_last_porty_entry_dataframe = concatted_first_and_last_porty_entry_dataframe.drop(concatted_first_and_last_porty_entry_dataframe.filter(regex='_y$').columns.tolist(), axis=1)

In [45]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone
0,60.01,71.25,71.81,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,25.26,25.30
1,140.01,145.56,145.65,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,137.81,139.58
2,230.01,250.18,250.55,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,221.72,221.91
3,310.01,311.58,311.87,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,307.16,308.42
4,385.01,386.07,386.16,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,384.48,384.67


In [46]:
concatted_first_and_last_porty_entry_dataframe.shape

(3048, 8)

- Getting all the rows that have port entry to port exit time that overlaps with the tone playing
    - This would be the port entries that came before the tone, but had a port exit after

In [47]:
port_entries_that_overlap = concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["port_exit_after_last_port_entry_before_tone"] >= concatted_first_and_last_porty_entry_dataframe["current_tone_time"]]

In [48]:
port_entries_that_overlap.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone
13,1240.01,1244.37,1253.24,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,1239.62,1241.43
15,1415.01,1422.51,1422.53,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,1413.82,1415.54
22,2055.01,2055.09,2055.12,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,2054.92,2055.03
23,2130.01,2130.44,2132.02,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,2129.99,2130.42
24,2230.01,2233.19,2233.25,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,2229.97,2233.15


In [49]:
port_entries_that_overlap.shape

(871, 8)

# Getting the Latency for Each Session of a Tone Playing

- To calculate latency, we would just subtract the time of the first port entry to that of the tone playing

In [50]:
concatted_first_and_last_porty_entry_dataframe["latency"] = concatted_first_porty_entry_dataframe["first_port_entry_after_tone"] - concatted_first_and_last_porty_entry_dataframe["current_tone_time"]

In [51]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency
0,60.01,71.25,71.81,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,25.26,25.30,11.24
1,140.01,145.56,145.65,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,137.81,139.58,5.55
2,230.01,250.18,250.55,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,221.72,221.91,20.17
3,310.01,311.58,311.87,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,307.16,308.42,1.57
4,385.01,386.07,386.16,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,384.48,384.67,1.06


- For all the latencies that are are greater than 30 seconds, we changed the latency to 30 seconds

In [52]:
concatted_first_and_last_porty_entry_dataframe["latency_adjusted_greater_than_30"] = concatted_first_and_last_porty_entry_dataframe["latency"].apply(lambda x: 30 if x >= 30 else x)

In [53]:
concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["latency"] >= 30].head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30
82,60.01,120.15,125.48,./example_recordings\2022-05-04_08h43m_Subject...,20220504,2.2,36.95,39.19,60.14,30.0
83,140.01,172.87,173.39,./example_recordings\2022-05-04_08h43m_Subject...,20220504,2.2,120.15,125.48,32.86,30.0
87,485.01,515.14,518.69,./example_recordings\2022-05-04_08h43m_Subject...,20220504,2.2,482.07,482.31,30.13,30.0
92,940.01,988.81,988.97,./example_recordings\2022-05-04_08h43m_Subject...,20220504,2.2,913.81,914.17,48.80,30.0
106,2230.01,2292.86,2293.07,./example_recordings\2022-05-04_08h43m_Subject...,20220504,2.2,2208.25,2208.75,62.85,30.0


# Seeing which port entries and port exit durations overlap with the tone

- Making a column that states whether or not the last port entry before the tone has a port exit after the tone. This would mean that they are overlapping.

In [54]:
concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"] = concatted_first_and_last_porty_entry_dataframe.apply(lambda row: True if row["port_exit_after_last_port_entry_before_tone"] >= row["current_tone_time"] else False, axis=1)


In [55]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone
0,60.01,71.25,71.81,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,25.26,25.30,11.24,11.24,False
1,140.01,145.56,145.65,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,137.81,139.58,5.55,5.55,False
2,230.01,250.18,250.55,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,221.72,221.91,20.17,20.17,False
3,310.01,311.58,311.87,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,307.16,308.42,1.57,1.57,False
4,385.01,386.07,386.16,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,384.48,384.67,1.06,1.06,False


- Comparing the latencies of the first port entry after the tone between these two groups:
    - 1. Those with the last port entry before the tone that has a port exit after the tone("overlapping" group)
    - 2. Those with the last port entry before the tone that has a port exit before the tone("nonoverlapping" group)

In [56]:
overlapping_df = concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"]]

In [57]:
non_overlapping_df = concatted_first_and_last_porty_entry_dataframe[~concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"]]

- Number of rows of overlapping and nonoverlapping last port entries before the tone

In [58]:
overlapping_df.shape

(871, 11)

In [59]:
non_overlapping_df.shape

(2177, 11)

- Latency between the tone playing and the first port entry after the tone

In [60]:
overlapping_df["latency"].mean()

13.85572413793093

In [61]:
non_overlapping_df["latency"].mean()

15.379288389513018

- Latency between the tone playing and the first port entry after the tone. This had been adjusted so that all latencies that are greater than 30 seconds were adjusted to be just 30 seconds.

In [62]:
overlapping_df["latency_adjusted_greater_than_30"].mean()

12.90272413793094

In [63]:
non_overlapping_df["latency_adjusted_greater_than_30"].mean()

6.909227528089803

# Adjusting the Latencies if the Port Entries/Exit Overlap With the Tone Times

- For all the rows with overlapping last port entries before the tone, we will adjust the latency of the tone to the first port entry after the tone to 0. (Because the first port entry after the tone would be considered to be the same as the last port entry before the tone)

In [64]:
concatted_first_and_last_porty_entry_dataframe["latency_adjusted_overlap"] = concatted_first_and_last_porty_entry_dataframe.apply(lambda row: 0 if row["overlapping_port_entry_with_tone"] else row["latency_adjusted_greater_than_30"], axis=1)

In [65]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap
0,60.01,71.25,71.81,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,25.26,25.30,11.24,11.24,False,11.24
1,140.01,145.56,145.65,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,137.81,139.58,5.55,5.55,False,5.55
2,230.01,250.18,250.55,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,221.72,221.91,20.17,20.17,False,20.17
3,310.01,311.58,311.87,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,307.16,308.42,1.57,1.57,False,1.57
4,385.01,386.07,386.16,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,384.48,384.67,1.06,1.06,False,1.06


- All the rows with overlapping last port entries before the tone

In [66]:
concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"]].head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap
13,1240.01,1244.37,1253.24,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,1239.62,1241.43,4.36,4.36,True,0.0
15,1415.01,1422.51,1422.53,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,1413.82,1415.54,7.50,7.50,True,0.0
22,2055.01,2055.09,2055.12,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,2054.92,2055.03,0.08,0.08,True,0.0
23,2130.01,2130.44,2132.02,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,2129.99,2130.42,0.43,0.43,True,0.0
24,2230.01,2233.19,2233.25,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,2229.97,2233.15,3.18,3.18,True,0.0


## Labeling if a row's latency is less than 10 seconds or not

- We will be getting the ratio of latencies(from the time that the tone played, to the first port entry after the tone) that are less than 10 seconds for each session. So we will label all the adjusted latencies that are less than 10 seconds as `True`, and the those with latencies that are greater than 10 seconds as `False` 

In [67]:
concatted_first_and_last_porty_entry_dataframe["adjusted_30_second_latency_less_than_10_seconds_ratio"] = concatted_first_and_last_porty_entry_dataframe["latency_adjusted_greater_than_30"].apply(lambda x: True if x <= 10 else False)

In [68]:
concatted_first_and_last_porty_entry_dataframe["overlap_adjusted_latency_less_than_10_seconds"] = concatted_first_and_last_porty_entry_dataframe["latency_adjusted_overlap"].apply(lambda x: True if x <= 10 else False)

In [69]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap,adjusted_30_second_latency_less_than_10_seconds_ratio,overlap_adjusted_latency_less_than_10_seconds
0,60.01,71.25,71.81,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,25.26,25.30,11.24,11.24,False,11.24,False,False
1,140.01,145.56,145.65,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,137.81,139.58,5.55,5.55,False,5.55,True,True
2,230.01,250.18,250.55,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,221.72,221.91,20.17,20.17,False,20.17,False,False
3,310.01,311.58,311.87,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,307.16,308.42,1.57,1.57,False,1.57,True,True
4,385.01,386.07,386.16,./example_recordings\2022-05-04_08h42m_Subject...,20220504,1.1,384.48,384.67,1.06,1.06,False,1.06,True,True


# Making Dataframes that have Each Subject as the Row and Each Date as the Column

- Pivot plots allow us to make columns out of each unique entry in a selected column. (For this, it will be the date column that we will turn into columns) The index for the rows will be each unique entry in another column. (For this, it will be the subject's ID) And the values will be the aggregate of the values that have both the values in the previous two selected columns. (For this it will be the latency columns and we will take the mean)
    - https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

- Making a folder for all the pivotplots

# NOTE: If you decide to change the cage numbers or the dates used from the original ones from the imported dataframes, you will need to change the subsequent path to the cage number and dates you are using

In [70]:
first_port_entry_to_tone_latency_output_directory = "./proc/subject_rows_with_date_columns_dataframes/first_port_entry_to_tone_latency/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [71]:
first_port_entry_to_tone_latency_output_directory

'./proc/subject_rows_with_date_columns_dataframes/first_port_entry_to_tone_latency/cage_1_2_date_20220504_20220516'

In [72]:
os.makedirs(first_port_entry_to_tone_latency_output_directory, exist_ok=True)

In [73]:
latencies_less_than_10_seconds_ratio_output_directory = "./proc/subject_rows_with_date_columns_dataframes/latencies_less_than_10_seconds_ratio/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [74]:
latencies_less_than_10_seconds_ratio_output_directory

'./proc/subject_rows_with_date_columns_dataframes/latencies_less_than_10_seconds_ratio/cage_1_2_date_20220504_20220516'

In [75]:
os.makedirs(latencies_less_than_10_seconds_ratio_output_directory, exist_ok=True)

## Making a pivot plot with the original latency

# TODO: Make new folders for each type

In [76]:
all_latency_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency', \
                                        index=['subject'], columns=['date'], aggfunc=np.mean)


In [77]:
all_latency_pivot_plot.head()

date,20220504,20220505,20220506,20220507,20220508,20220509,20220510,20220511,20220512,20220516
subject,,,,,,,,,,
1.1,6.046829,11.537750,5.956341,3.59800,10.863590,7.18400,9.31450,9.92700,NaN,8.523659
1.2,25.471538,13.112195,7.628250,5.64800,4.987692,4.09525,9.06400,6.92650,18.89000,10.487000
1.3,36.300526,373.844750,14.234500,7.00100,8.922821,8.97225,8.34525,7.85100,16.60550,13.887250
1.4,10.832683,125.872000,29.977805,12.33400,6.594103,5.93475,9.34625,19.38650,NaN,12.103659
2.1,7.548500,18.269268,5.618500,3.92925,6.335897,5.16775,6.82400,4.94175,17.10825,9.888500


- Getting the cage numbers for each subject and then making a dictionary out of it
    - This will be used to add the cage information to the pivot plots

In [78]:
# Converting the cage numbers from floats to integers
# When the metadata is imported from the csv files, the cage number is imported as a float
metadata_df["cage"] = metadata_df["cage"].astype(int)

In [79]:
subject_to_cage = pd.Series(metadata_df["cage"].values,index=metadata_df["Subject"].values).to_dict()

In [80]:
subject_to_cage

{1.1: 1, 1.4: 1, 2.2: 1, 2.3: 1, 1.2: 2, 1.3: 2, 2.1: 2, 2.4: 2}

- Adding the cage information with the dictionary

In [81]:
all_latency_pivot_plot["cage"] = all_latency_pivot_plot.index.map(subject_to_cage)

In [82]:
all_latency_pivot_plot.head()

date,20220504,20220505,20220506,20220507,20220508,20220509,20220510,20220511,20220512,20220516,cage
subject,,,,,,,,,,,
1.1,6.046829,11.537750,5.956341,3.59800,10.863590,7.18400,9.31450,9.92700,NaN,8.523659,1
1.2,25.471538,13.112195,7.628250,5.64800,4.987692,4.09525,9.06400,6.92650,18.89000,10.487000,2
1.3,36.300526,373.844750,14.234500,7.00100,8.922821,8.97225,8.34525,7.85100,16.60550,13.887250,2
1.4,10.832683,125.872000,29.977805,12.33400,6.594103,5.93475,9.34625,19.38650,NaN,12.103659,1
2.1,7.548500,18.269268,5.618500,3.92925,6.335897,5.16775,6.82400,4.94175,17.10825,9.888500,2


In [83]:
os.path.join(first_port_entry_to_tone_latency_output_directory, "subject_to_date_original_latency_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date))

'./proc/subject_rows_with_date_columns_dataframes/first_port_entry_to_tone_latency/cage_1_2_date_20220504_20220516\\subject_to_date_original_latency_cage_1_2_date_20220504_20220516.csv'

In [93]:
all_latency_pivot_plot.to_csv(os.path.join(first_port_entry_to_tone_latency_output_directory, "latency_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
all_latency_pivot_plot.to_excel(os.path.join(first_port_entry_to_tone_latency_output_directory, "latency_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [143]:
file_path = "subject_to_date_original_latency_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)
test_path = u"\\\\?\\" + os.path.abspath(os.path.join(first_port_entry_to_tone_latency_output_directory, file_path))

In [145]:
all_latency_pivot_plot.to_csv(test_path)

In [84]:
all_latency_pivot_plot.to_csv(os.path.join(first_port_entry_to_tone_latency_output_directory, "subject_to_date_original_latency_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
all_latency_pivot_plot.to_excel(os.path.join(first_port_entry_to_tone_latency_output_directory, "subject_to_date_original_latency_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

FileNotFoundError: [Errno 2] No such file or directory: './proc/subject_rows_with_date_columns_dataframes/first_port_entry_to_tone_latency/cage_1_2_date_20220504_20220516\\subject_to_date_original_latency_cage_1_2_date_20220504_20220516.csv'

## Making a pivot plot with the latency that were greater than 30 seconds being adjusted 

In [ ]:
latency_greater_than_30_seconds_adjusted_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency_adjusted_greater_than_30', index=['subject'],

                    columns=['date'], aggfunc=np.mean)


In [ ]:
latency_greater_than_30_seconds_adjusted_pivot_plot["cage"] = latency_greater_than_30_seconds_adjusted_pivot_plot.index.map(subject_to_cage)

In [ ]:
latency_greater_than_30_seconds_adjusted_pivot_plot.head()

In [ ]:
latency_greater_than_30_seconds_adjusted_pivot_plot.to_csv(os.path.join(first_port_entry_to_tone_latency_output_directory, "subject_to_date_latency_greater_than_30_seconds_adjusted_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
latency_greater_than_30_seconds_adjusted_pivot_plot.to_excel(os.path.join(first_port_entry_to_tone_latency_output_directory, "subject_to_date_latency_greater_than_30_seconds_adjusted_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

## Making a pivot plot with adjusted latencies that overlapped with a tone playing

In [ ]:
latency_that_overlaps_with_tone_adjusted_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency_adjusted_overlap', index=['subject'],

                    columns=['date'], aggfunc=np.mean)


In [ ]:
latency_that_overlaps_with_tone_adjusted_pivot_plot["cage"] = latency_that_overlaps_with_tone_adjusted_pivot_plot.index.map(subject_to_cage)

In [ ]:
latency_that_overlaps_with_tone_adjusted_pivot_plot.head()

In [ ]:
latency_that_overlaps_with_tone_adjusted_pivot_plot.to_csv(os.path.join(first_port_entry_to_tone_latency_output_directory, "subject_to_date_latency_that_overlaps_with_tone_adjusted_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
latency_that_overlaps_with_tone_adjusted_pivot_plot.to_excel(os.path.join(first_port_entry_to_tone_latency_output_directory, "subject_to_date_latency_that_overlaps_with_tone_adjusted_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

## Making a pivot plot with the ratio of original latencies that are less than 10 seconds

In [ ]:
original_less_than_10_latency_df = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='adjusted_30_second_latency_less_than_10_seconds_ratio', index=['subject'],

                    columns=['date'], aggfunc=np.mean)


In [ ]:
original_less_than_10_latency_df["cage"] = original_less_than_10_latency_df.index.map(subject_to_cage)

In [ ]:
original_less_than_10_latency_df

In [ ]:
original_less_than_10_latency_df.to_csv(os.path.join(latencies_less_than_10_seconds_ratio_output_directory, "subject_to_date_less_than_10_seconds_original_latencies_ratios_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
original_less_than_10_latency_df.to_excel(os.path.join(latencies_less_than_10_seconds_ratio_output_directory, "subject_to_date_less_than_10_seconds_original_latencies_ratios_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

## Making a pivot plot with the ratio of overlap adjusted latencies that are less than 10 seconds

In [ ]:
overlap_adjusted_less_than_10_latency_df = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='overlap_adjusted_latency_less_than_10_seconds', index=['subject'],

                    columns=['date'], aggfunc=np.mean)


In [ ]:
overlap_adjusted_less_than_10_latency_df["cage"] = overlap_adjusted_less_than_10_latency_df.index.map(subject_to_cage)

In [ ]:
overlap_adjusted_less_than_10_latency_df

In [ ]:
overlap_adjusted_less_than_10_latency_df.to_csv(os.path.join(latencies_less_than_10_seconds_ratio_output_directory, "subject_to_date_less_than_10_seconds_overlap_adjusted_latencies_ratios_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
overlap_adjusted_less_than_10_latency_df.to_excel(os.path.join(latencies_less_than_10_seconds_ratio_output_directory, "subject_to_date_less_than_10_seconds_overlap_adjusted_latencies_ratios_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

# Plotting the latencies

## Getting the averages for all the trials in one subject and one day

- We will plot a line for each subject. With the X-axis being the days since the first session. And the Y-axis is the latency values averaged across all the trials for one recording session
- Grouping all the rows with the same subject and date(aka, all the trials in one session). And then getting the mean for each value

In [ ]:
grouped_averaged_first_porty_entry_dataframe = concatted_first_and_last_porty_entry_dataframe.groupby(by=["file_path", "date", "subject"]).mean()

In [ ]:
grouped_averaged_first_porty_entry_dataframe.head()

## Reformatting the Dataframe for plotting

- Resetting the index so that we can use the data as a value

In [ ]:
grouped_averaged_first_porty_entry_dataframe = grouped_averaged_first_porty_entry_dataframe.reset_index()

In [ ]:
grouped_averaged_first_porty_entry_dataframe.head()

- Turning the date column which is a string, into an integer
    - This will allow us to use the date as the X-axis for plotting

In [ ]:
grouped_averaged_first_porty_entry_dataframe["date_int"] = grouped_averaged_first_porty_entry_dataframe["date"].astype(int)

In [ ]:
grouped_averaged_first_porty_entry_dataframe.head()

- Adding the cage information
    - We will make one plot for each cage

In [ ]:
grouped_averaged_first_porty_entry_dataframe["cage"] = grouped_averaged_first_porty_entry_dataframe["subject"].map(subject_to_cage).astype(int)

In [ ]:
grouped_averaged_first_porty_entry_dataframe.head()

## Plotting the latencies that were adjusted if they greater than 30 seconds

In [ ]:
original_average_latency_output_directory = "./proc/plots/original_average_latency_plots/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [ ]:
original_average_latency_output_directory

In [ ]:
os.makedirs(original_average_latency_output_directory, exist_ok=True)

In [ ]:
# Plotting for each cage
for cage in grouped_averaged_first_porty_entry_dataframe["cage"].unique():
    fig, ax = plt.subplots()
    # Getting all the rows with the current cage number
    cage_df = grouped_averaged_first_porty_entry_dataframe[grouped_averaged_first_porty_entry_dataframe["cage"] == cage]
    
    # Plotting a line for each subject
    for subject in cage_df["subject"].unique():
        # Getting all the rows with the current subject
        subject_df = cage_df[cage_df["subject"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        ax.plot(subject_df["date_int"] - subject_df["date_int"].min() + 1, subject_df["latency_adjusted_greater_than_30"], '-o', label=subject)

    # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
    ax.set_ylim(0, 30)
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("The Days After the First Session")
    ax.set_ylabel("Adjusted Average Latency of First Entry to Tone Onset")
    ax.set_title("Latency of Port Entry to Tone: Cage {}".format(cage))
    # To show the legend
    ax.legend()
    file_name = "original_average_latency_plot_cage_{}_date_{}_{}.png".format(cage, earliest_date, latest_date)
    plt.savefig(os.path.join(original_average_latency_output_directory, file_name))

## Plotting the original ratio of latencies that are less than 10 seconds

In [ ]:
less_than_10_second_original_latency_ratio_plots_output_directory = "./proc/plots/less_than_10_second_original_latency_ratio/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [ ]:
less_than_10_second_original_latency_ratio_plots_output_directory

In [ ]:
os.makedirs(less_than_10_second_original_latency_ratio_plots_output_directory, exist_ok=True)

In [ ]:
# Plotting for each cage
for cage in grouped_averaged_first_porty_entry_dataframe["cage"].unique():
    fig, ax = plt.subplots()
    # Getting all the rows with the current cage number
    cage_df = grouped_averaged_first_porty_entry_dataframe[grouped_averaged_first_porty_entry_dataframe["cage"] == cage]
    
    # Plotting a line for each subject
    for subject in cage_df["subject"].unique():
        # Getting all the rows with the current subject
        subject_df = cage_df[cage_df["subject"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        ax.plot(subject_df["date_int"] - subject_df["date_int"].min() + 1, subject_df["adjusted_30_second_latency_less_than_10_seconds_ratio"], '-o', label=subject)
    # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
    ax.set_ylim(0, 1)
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("The Days After the First Session")
    ax.set_ylabel("Proportion of Latencies")
    ax.set_title("Less Than 10 Seconds Latencies from Tone Onset: Cage {}".format(cage))
    # To show the legend
    ax.legend()
    
    file_name = "less_than_10_second_original_latency_ratio_plot_cage_{}_date_{}_{}.png".format(cage, earliest_date, latest_date)
    plt.savefig(os.path.join(less_than_10_second_original_latency_ratio_plots_output_directory, file_name))


# Plotting the Adjusted Latencies for Entries that Overlap with the Tone

## Plotting the Latencies 

In [ ]:
overlap_adjusted_average_latency_plots_output_directory = "./proc/plots/overlap_adjusted_average_latency_plots/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [ ]:
overlap_adjusted_average_latency_plots_output_directory

In [ ]:
os.makedirs(overlap_adjusted_average_latency_plots_output_directory, exist_ok=True)

In [ ]:
# Plotting for each cage
for cage in grouped_averaged_first_porty_entry_dataframe["cage"].unique():
    fig, ax = plt.subplots()
    # Getting all the rows with the current cage number
    cage_df = grouped_averaged_first_porty_entry_dataframe[grouped_averaged_first_porty_entry_dataframe["cage"] == cage]
    
    # Plotting a line for each subject
    for subject in cage_df["subject"].unique():
        # Getting all the rows with the current subject
        subject_df = cage_df[cage_df["subject"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        ax.plot(subject_df["date_int"] - subject_df["date_int"].min() + 1, subject_df["latency_adjusted_overlap"], '-o', label=subject)

    # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
    ax.set_ylim(0, 30)
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("The Days After the First Session")
    ax.set_ylabel("Proportion of Latencies")
    ax.set_title("Overlap Adjusted Latency of Port Entry to Tone: Cage {}".format(cage))
    # To show the legend
    ax.legend()
    file_name = "overlap_adjusted_average_latency_plot_cage_{}_date_{}_{}.png".format(cage, earliest_date, latest_date)

    plt.savefig(os.path.join(overlap_adjusted_average_latency_plots_output_directory, file_name))


# Plotting the ratio of overlap adjusted latencies that are less than 10 seconds

In [ ]:
less_than_10_second_overlap_adjusted_latency_ratio_output_directory = "./proc/plots/less_than_10_second_overlap_adjusted_latency_ratio/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [ ]:
less_than_10_second_overlap_adjusted_latency_ratio_output_directory

In [ ]:
os.makedirs(less_than_10_second_overlap_adjusted_latency_ratio_output_directory, exist_ok=True)

In [ ]:
# Plotting for each cage
for cage in grouped_averaged_first_porty_entry_dataframe["cage"].unique():
    fig, ax = plt.subplots()
    # Getting all the rows with the current cage number
    cage_df = grouped_averaged_first_porty_entry_dataframe[grouped_averaged_first_porty_entry_dataframe["cage"] == cage]
    
    # Plotting a line for each subject
    for subject in cage_df["subject"].unique():
        # Getting all the rows with the current subject
        subject_df = cage_df[cage_df["subject"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        ax.plot(subject_df["date_int"] - subject_df["date_int"].min() + 1, subject_df["overlap_adjusted_latency_less_than_10_seconds"], '-o', label=subject)

    # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
    ax.set_ylim(0, 1)
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("The Days After the First Session")
    ax.set_ylabel("Proportion of Latencies")
    ax.set_title("Overlap Adjusted Less Than 10sec Latencies from Tone: Cage {}".format(cage))
    # To show the legend
    ax.legend()
    
    file_name = "less_than_10_second_overlap_adjusted_latency_ratio_plot_cage{}_date_{}_{}.png".format(cage, earliest_date, latest_date)
    plt.savefig(os.path.join(less_than_10_second_overlap_adjusted_latency_ratio_output_directory, file_name))
